In [1]:
import msoffcrypto
import io
import re
import pandas as pd
from pyxlsb import open_workbook
import os
import numpy as np
import calendar
import glob
from pathlib import Path

pd.set_option("display.max_columns", None)

In [2]:
password = "CRQAD_amf_2025!"

files = glob.glob("C:/Users/Home/Downloads/Capstone/BScore/**/DW_BSCORE-CLG_**.xlsb", 
                   recursive = True)

dfs = {}
for file in files:
    decrypted = io.BytesIO()
    is_encrypted = False

    try: # if file is encrypted
        with open(file, "rb") as f:
            office_file = msoffcrypto.OfficeFile(f)
            office_file.load_key(password=password)
            office_file.decrypt(decrypted)
        is_encrypted = True
        
    except Exception as e:
        decrypted = file
        
    df = []
    with open_workbook(decrypted) as wb:
        with wb.get_sheet(1) as sheet:  # Read first sheet
            for row in sheet.rows():
                df.append([item.v for item in row])

    df = pd.DataFrame(df[1:], columns=df[0])  # Convert to pandas DataFrame
    dfs[file] = df  # Store DataFrame with file name as key

    if is_encrypted:
        print(f"Successfully loaded decrypted: {file}")
    else:
        print(f"Successfully loaded unencrypted: {file}")

Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_01302015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_02272015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_03312015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_04302015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_05292015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_06302015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_07312015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_08282015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/BScore\2015\DW_BSCORE-CLG_09302015.xlsb
Successfully loaded decrypted: C:/Users/Home/Downloads/Capstone/

In [3]:
folder_path = "C:/Users/Home/Downloads/Capstone/BScore/**/DW_BSCORE-CLG_**.xlsb"
result = [re.search(r'\d{8}', x).group() for x in glob.glob(folder_path)]
month_year = [f"{date[:2]} {date[4:]}" for date in result]

dfs_new = []

for mon_yr, tup in zip(month_year,dfs.items()):
    file = tup[0] 
    df = tup[1] 
    
    df["FROM_AS_OF_DATE"] = pd.to_datetime(df["FROM_AS_OF_DATE"], format="%m/%d/%Y")
    df["TO_AS_OF_DATE"] = pd.to_datetime(df["TO_AS_OF_DATE"], format="%m/%d/%Y")
    df["ISSUE_DATE"] = pd.to_datetime(df["ISSUE_DATE"], format="%m/%d/%Y")
    df["MATURITY_DATE"] = pd.to_datetime(df["MATURITY_DATE"], format="%m/%d/%Y")

    df_long = df.iloc[:, np.r_[0:2,3:12]].copy()
    df_long["ACTUAL_AMORT_RATIO"] = (df_long["PRINCIPAL_PYMT"] + df_long["INTEREST_PYMT"]) / (df_long["AMORT"])
    df_long["REMAINING_LOAN_RATIO"] = df_long["CURRENT_LOAN_BAL"] / df_long["ORIG_LOAN_BAL"]
    df_long = df_long.rename(columns={'ID_NUMBER':'ILS_ACCT_NUM'})
    
    # for x in month_year:
    df_long = df_long.rename(columns = {
        "ACTUAL_AMORT_RATIO": f"ACTUAL_AMORT_RATIO {mon_yr}", 
        "REMAINING_LOAN_RATIO" : f"REMAINING_LOAN_RATIO {mon_yr}",
        "AMORT": f"AMORT {mon_yr}",
        "CURRENT_LOAN_BAL": f"CURRENT_LOAN_BAL {mon_yr}",
        "INTEREST_PYMT": f"INTEREST_PYMT {mon_yr}",
        "PRINCIPAL_PYMT": f"PRINCIPAL_PYMT {mon_yr}",
    })
    
    dfs_new = dfs_new + [df_long]

In [4]:
merged = pd.DataFrame()

for df in dfs_new:
    if merged.empty: 
        merged = df.iloc[:, np.r_[2,7:13]]
    else: 
        merged = merged.merge(
            df.iloc[:, np.r_[2,7:13]],
            on = ["ILS_ACCT_NUM"],
            how="outer"
        )

merged.to_csv("bscore_final.csv", index=False)